In [1]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'../resources/fonts/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumGothic'}) # 폰트 설정
plt.rc('font', family='NanumGothic')
import seaborn as sns
import ast

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

from sklearn.metrics.pairwise import cosine_similarity 

In [2]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
movie_path = '../datasets/movies_metadata.csv'
rating_path = '../datasets/ratings_small.csv'

movie_dt = pd.read_csv(movie_path)
rating_dt = pd.read_csv(rating_path)

In [3]:
movie_dt.shape

(45466, 24)

In [4]:
#결측치가 35,000개 이상인건 의미없다고 판단 (영화가 45,000개인데 35,000개면 75% 결측)
drop_column_list = list(movie_dt.columns[movie_dt.isnull().sum()<=35000])
movie_dt = movie_dt[drop_column_list]

#status가 released가 아닌 영화들은 볼수 없으니 제외
#데이터셋이 과거의 자료여서 post production, In Production 등 production중인 영화들은 이미 개봉했을 수 있지만
#무비 데이터를 최신으로 변경하기 전까지는 현재 데이터셋에선 우선 제거하는걸로
movie_dt = movie_dt[movie_dt['status'] == 'Released'].dropna(subset=['status'])

#지금단계에서 필요없어보이는 column 제거
movie_dt = movie_dt.drop(['overview', 'poster_path', 'tagline', 'status', 'spoken_languages'], axis=1)

# 최종적으로 남은 열 확인
remaining_columns = movie_dt.columns
print(remaining_columns)

Index(['adult', 'budget', 'genres', 'id', 'imdb_id', 'original_language',
       'original_title', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime', 'title',
       'video', 'vote_average', 'vote_count'],
      dtype='object')


In [5]:
movie_dt['popularity'] = movie_dt['popularity'].astype(float)

In [6]:
def extract_genre_names(genres_string):
    genres_list = ast.literal_eval(genres_string)  # 문자열을 파이썬 객체로 변환
    genre_names = [genre['name'] for genre in genres_list]  # 이름만 추출
    return ', '.join(genre_names)  # 쉼표로 연결

# 새로운 열 추가
movie_dt['genre_names'] = movie_dt['genres'].apply(extract_genre_names)

In [7]:
# 새로운 열이 제대로 추가되었는지 상위 5개의 데이터를 확인
print(movie_dt[['title', 'genres', 'genre_names']].head())

                         title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                  genre_names  
0   Animation, Comedy, Family  
1  Adventure, Fantasy, Family  
2             Romance, Comedy  
3      Comedy, Drama, Romance  
4                      Comedy  


## 신규 유저

In [8]:
# 기존 유저인지 신규 유저인지 판별하는 함수
def check_user(user_id, rating_dt):
    if user_id in rating_dt['userId'].unique():
        return "existing"
    else:
        return "new"

# 신규 유저에게 userId 부여
def make_new_userId(rating_dt):
    return rating_dt['userId'].max() + 1

# 사용자 입력 받기
def get_user_id(rating_dt):
    user_input = input("본인의 userId를 입력하세요 (신규 유저일 경우 'new'를 입력하세요): ")

    # 기존 유저인지 신규 유저인지 판별
    if user_input.lower() == 'new':
        user_id = make_new_userId(rating_dt)
        print(f"신규 유저입니다. 부여된 userId는 {user_id}입니다.")
    else:
        user_id = int(user_input)
        user_type = check_user(user_id, rating_dt)
        if user_type == "existing":
            print(f"기존 유저입니다. userId는 {user_id}입니다.")
        else:
            print(f"입력한 userId({user_id})는 존재하지 않습니다. 새로 userId를 부여합니다.")
            user_id = make_new_userId(rating_dt)
            print(f"부여된 userId는 {user_id}입니다.")
    
    return user_id


# user_id 값 결정 및 저장
user_id = get_user_id(rating_dt)


입력한 userId(789)는 존재하지 않습니다. 새로 userId를 부여합니다.
부여된 userId는 672입니다.


## Cold start

- 20개 장르 중 5개 장르 선택
- 20개의 영화 중 5개 영화 선택


- 재미있게 본 영화 평점 4점 책정

- ratings_small.csv에 새 유저의 userId, movieId, rating 추가
- 기존 유저화 되면서 기존 유저와 동일하게 추천 시스템 적용

In [9]:
# ','로 구분된 장르를 개별적으로 추출하여 리스트로 변환
unique_genres = movie_dt['genre_names'].dropna().str.split(', ').explode().unique()

# 빈 문자열('')을 제외
unique_genres = [genre for genre in unique_genres if genre != '']

# 사용자로부터 장르 입력 받기

print("20개의 장르 중 5개를 선택하세요. (쉼표로 구분)")
print(unique_genres)


20개의 장르 중 5개를 선택하세요. (쉼표로 구분)
['Animation', 'Comedy', 'Family', 'Adventure', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'History', 'Science Fiction', 'Mystery', 'War', 'Foreign', 'Music', 'Documentary', 'Western', 'TV Movie']


In [13]:
user_input = input("선택한 장르를 입력하세요 (예: Action, Drama, Comedy, War, History): ")

# Step 5: 입력값 처리 (쉼표로 구분하여 리스트로 변환)
selected_genres = [genre.strip() for genre in user_input.split(',')]

# Step 6: 선택된 장르를 출력
print(f"선택된 장르: {selected_genres}")

# Step 7: 선택된 장르에 해당하는 영화 필터링 함수 정의
def filter_movies_by_genres(movie_metadata, selected_genres):
    return movie_metadata[movie_metadata['genre_names'].apply(lambda x: any(genre in x for genre in selected_genres))]

# Step 8: 선택된 장르에 해당하는 영화 필터링
filtered_movies = filter_movies_by_genres(movie_dt, selected_genres)

# Step 9: revenue 기준으로 내림차순 정렬하여 상위 20개 추천
top_movies_by_revenue = filtered_movies.nlargest(20, 'revenue')


# Step 10: 결과 확인
# print(top_movies_by_revenue[['title', 'genre_names', 'revenue']])

# Step 10: 결과 확인 - 영화 제목 결과
print("\n추천 영화 목록 (상위 20개):")
print(top_movies_by_revenue['title'])


선택된 장르: ['']

추천 영화 목록 (상위 20개):
14551                                           Avatar
26555                     Star Wars: The Force Awakens
1639                                           Titanic
17818                                     The Avengers
25084                                   Jurassic World
28830                                        Furious 7
26558                          Avengers: Age of Ultron
17437     Harry Potter and the Deathly Hallows: Part 2
22110                                           Frozen
42222                             Beauty and the Beast
43255                          The Fate of the Furious
20830                                       Iron Man 3
30700                                          Minions
26567                       Captain America: Civil War
17293                   Transformers: Dark of the Moon
7000     The Lord of the Rings: The Return of the King
19261                                          Skyfall
23617                  Transform

In [14]:
# Step 1: 사용자로부터 선택한 영화를 입력받기
user_input = input("재미있게 본 영화 ID를 선택하세요 (쉼표로 구분): ")

# Step 2: 입력값 처리 (쉼표로 구분하여 리스트로 변환)
selected_movies = [movie.strip() for movie in user_input.split(',')]
print(f"선택된 영화: {selected_movies}")

선택된 영화: ['7000', '17293', '30700']


In [15]:
# 신규 유저 ID 부여 (가정: rating_dt['userId'].max() + 1)

# 선택된 영화에 평점 4점 부여
new_rating = []
for movie_id in selected_movies:
    new_rating.append([user_id, movie_id, 4.0])  # 평점은 4점으로 고정

# 새로 추가할 rating 데이터프레임 생성
new_rating_dt = pd.DataFrame(new_rating, columns=['userId', 'movieId', 'rating'])

# 기존 ratings 데이터프레임 불러오기 
rating_dt = pd.read_csv(rating_path)

# 기존 ratings 데이터프레임에 신규 유저의 평점 추가
rating_dt = pd.concat([rating_dt, new_rating_dt], ignore_index=True)

# Step 4: 업데이트된 ratings 데이터를 CSV 파일로 저장
rating_dt.to_csv('ratings_small_updated.csv', index=False)

print("\n유저의 평점이 성공적으로 추가되었습니다.")


유저의 평점이 성공적으로 추가되었습니다.


## 모델 학습

In [16]:
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

# userId와 movieId를 정수형으로 변환
rating_dt['userId'] = rating_dt['userId'].astype(int)
rating_dt['movieId'] = rating_dt['movieId'].astype(int)

# CSR 형식으로 변환
user_item_matrix = csr_matrix((rating_dt['rating'], (rating_dt['userId'], rating_dt['movieId'])))



In [17]:

# ALS 모델 초기화
model = AlternatingLeastSquares(factors=10, regularization=0.1, iterations=20)
# 모델 학습
model.fit(user_item_matrix)


  0%|          | 0/20 [00:00<?, ?it/s]

In [18]:
#유저 아이디를 입력하면 다섯개의 추천 목록을 뽑아준다.
def recommend_movie_by_als(user_id, n_movies=5):
    recommendations = model.recommend(user_id, user_item_matrix[user_id], N=n_movies)
    for ids in recommendations[0]:
        movie_title = movie_dt.loc[movie_dt.index == ids, 'title'].values
        print(movie_title)
    return
    

In [19]:
recommend_movie_by_als(672)

['The American President']
['Vanya on 42nd Street']
['Blue in the Face']
['The Browning Version']
['Silver Bullet']
